# 6. Классификация текстов при помощи сверточных сетей

__Автор__: Никита Владимирович Блохин (NVBlokhin@fa.ru)

Финансовый университет, 2020 г.

In [111]:
import nltk
import torch
import torch.nn as nn

## 1. Представление и предобработка текстовых данных в виде последовательностей

1.1 Представьте первое предложение из строки `text` как последовательность из индексов слов, входящих в это предложение

In [112]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [113]:
text = text.lower()

alphabet = list(set(nltk.word_tokenize(text.replace(".", ""))))
word2index = {w: i for i, w in enumerate(alphabet)}

first_sentence = nltk.sent_tokenize(text)[0].replace(".", "")
[word2index[w] for w in nltk.word_tokenize(first_sentence)]

[2, 7, 14, 19, 17, 22, 20, 24]

1.2 Представьте первое предложение из строки `text` как последовательность векторов, соответствующих индексам слов. Для представления индекса в виде вектора используйте унитарное кодирование. В результате должен получиться двумерный тензор размера `количество слов в предложении` x `количество уникальных слов`

In [114]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [117]:
text = text.lower()

alphabet = list(set(nltk.word_tokenize(text.replace(".", ""))))
word2index = {w: i for i, w in enumerate(alphabet)}

first_sentence = nltk.sent_tokenize(text)[0].replace(".", "")
words = nltk.word_tokenize(first_sentence)

vectors = torch.zeros(len(words), len(alphabet))
indices = [(i, word2index[w]) for i, w in enumerate(words)]
vectors[list(zip(*indices))] = 1
vectors

tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1.]])

1.3 Решите задачу 1.2, используя модуль `nn.Embedding`

In [119]:
torch.manual_seed(0)

# попадание в размерность - уже успех
embeds = nn.Embedding(num_embeddings=len(alphabet), embedding_dim=len(alphabet))
indices = torch.tensor([word2index[w] for w in nltk.word_tokenize(first_sentence)])
embeds(indices)

tensor([[ 1.0554,  0.1778, -0.2303, -0.3918,  0.5433, -0.3952, -0.4462,  0.7440,
          1.5210,  3.4105, -1.5312, -1.2341,  1.8197, -0.5515, -0.5692,  0.9200,
          1.1108,  1.2899, -1.4782,  2.5672, -0.4731,  0.3356, -1.6293, -0.5497,
         -0.4798],
        [-1.3962, -0.0661, -0.3584, -1.5616, -0.3546,  1.0811,  0.1315,  1.5735,
          0.7814, -1.0787, -0.7209,  1.4708,  0.2756,  0.6668, -0.9944, -1.1894,
         -1.1959, -0.5596,  0.5335,  0.4069,  0.3946,  0.1715,  0.8760, -0.2871,
          1.0216],
        [ 1.9595, -1.1038,  0.5411,  1.5390,  1.0860,  1.2464,  0.1151,  1.6193,
          0.4637,  1.3007,  0.8732,  0.0651,  0.7732, -0.9701, -0.8877, -0.3183,
         -0.3344,  0.4543,  0.4990,  0.8780,  0.3894,  1.4625,  0.4795, -0.5334,
         -0.0347],
        [ 0.0358,  0.2160, -0.9161,  1.5599, -3.1537, -0.5611, -0.4303, -0.3332,
         -1.5464, -0.0147,  1.2251,  1.5936, -1.6315, -0.0569,  0.6297,  0.2712,
         -0.6860, -1.0918,  1.6797, -0.8808,  0.5800

## 2. Классификация фамилий по национальности (ConvNet)

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`. 

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса> 
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины

2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта: 
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [ ]:
class Vocab:
    pass

In [ ]:
class SurnamesDataset(Dataset):
    pass

## 3. Классификация обзоров на фильмы (ConvNet)

Датасет: https://disk.yandex.ru/d/tdinpb0nN_Dsrg

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста

2.2. Обучите классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding` 
    - подберите адекватную размерность вектора эмбеддинга: 
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%